# Points to be clarified

#### 1) To show the recommendations in DT, should we take the information from the solutions_summary.csv file ?

#### Answer: YES

| Solution | Comfort (%) | Energy (Wh) | Avg Temp (°C) | Avg RH (%) | Energy Savings (%) |
|----------|-------------|-------------|---------------|------------|--------------------|
| S1       | 100.00      | 2232.11     | 21.81         | 51.62      | 77.25              |
| S2       | 98.33       | 2387.57     | 20.61         | 53.74      | 75.66              |
| S3       | 95.83       | 2620.77     | 18.81         | 56.92      | 73.28              |

<ul><li>if you want to stay in comfort set: -> S1<ul><li>Internal Temperature: 21.81 °C</li><li>Relative Humidity: 51.62 %</li><li>Energy Saving: 77.25 %</li></ul><li>If you want to minimize energy consumption without taking into account comfort set -> S3</li><li>If you want to minimize energy consumption with an acceptable comfort set: -> S2</li></ul>

<hr>

#### 2) To show sPMV values in DT, should we take the information from the sPMV column of features_with_forecast.csv file?

#### Answer: sPMV is calculated using temperature and humidity

#### Do you think showing the current day values is enough?

#### Answer: next 24 hours for the forecasting and last week for the historical

| time                      | sPMV | baseline | flexibility_below | energy_average | t_r   | rh_r  | hour | day_of_week | is_weekend | forecasted_sPMV | co2_ppm |
|---------------------------|------|----------|-------------------|----------------|-------|-------|------|-------------|------------|-----------------|---------|
| 2024-07-04 21:00:00+00:00 | 0.68 | 133.825  | 123.98            | 128.9025       | 26.86 | 47.42 | 21   | 3           | 0          | 0.78            | 744.83  |
| 2024-07-04 22:00:00+00:00 | 0.73 | 102.19   | 101.08            | 101.635        | 27.00 | 49.19 | 22   | 3           | 0          | 0.807           | 633.086 |


<hr>

#### 3) Dataset Creation

<table>
      <tr>
        <th>Column</th>
        <th>Description</th>
        <th>Unit/Format</th>
        <th>Question</th>
        <th>Answer</th>
      </tr>
      <tr>
        <td>Timestamp</td>
        <td>Date and time of the record</td>
        <td>YYYY-MM-DD HH:MM:SS</td>
        <td></td>
        <td></td>
      </tr>
      <tr>
        <td>sPMV</td>
        <td>Simplified Predicted Mean Vote value</td>
        <td>Float (-3 to +3)</td>
        <td>How to get this info?</td>
        <td>function output</td>
      </tr>
      <tr>
        <td>Baseline energy</td>
        <td>Reference energy consumption</td>
        <td>Wh</td>
        <td>How to get this info?</td>
        <td>From NTUA</td>
      </tr>
      <tr>
        <td>Flexibility below</td>
        <td>Energy flexibility range below baseline</td>
        <td>Wh</td>
        <td>How to get this info?</td>
        <td>From NTUA</td>
      </tr>
      <tr>
        <td>Energy average</td>
        <td>Average energy consumption</td>
        <td>Wh</td>
        <td>How to get this info?</td>
        <td>Baseline energy and flexibility below average value.</td>
      </tr>
      <tr>
        <td>Indoor temperature</td>
        <td>Indoor air temperature (t_r)</td>
        <td>°C</td>
        <td>From ISRAA(Domx) APIs? Which are the sensors? We need to group the sensors values?</td>
        <td>we have the list of the sensors, 1 sensor for each apartment.</td>
      </tr>
      <tr>
        <td>Indoor humidity</td>
        <td>Indoor relative humidity (rh_r)</td>
        <td>%</td>
        <td>From ISRAA(Domx) APIs? Which are the sensors? We need to group the sensors values?</td>
        <td>we have the list of the sensors, 1 sensor for each apartment.</td>
      </tr>
      <tr>
        <td>Hour</td>
        <td>Hour of day</td>
        <td>0-23</td>
        <td></td>
        <td></td>
      </tr>
      <tr>
        <td>Day of week</td>
        <td>Day of the week</td>
        <td>0-6 (0=Monday, 6=Sunday)</td>
        <td></td>
        <td></td>
      </tr>
      <tr>
        <td>Is weekend</td>
        <td>Whether the day is a weekend</td>
        <td>0 or 1</td>
        <td></td>
        <td></td>
      </tr>
      <tr>
        <td>Forecasted sPMV</td>
        <td>Predicted sPMV from LSTM model</td>
        <td>Float (-3 to +3)</td>
        <td></td>
        <td></td>
      </tr>
      <tr>
        <td>CO₂ concentration</td>
        <td>Carbon dioxide level (if available)</td>
        <td>ppm</td>
        <td>From ISRAA(Domx) APIs? Which are the sensors? We need to group the sensors values?</td>
        <td>we have the list of the sensors, 1 sensor for each apartment.</td>
      </tr>
    </table>

#### Answer: baseline energy and flexibility below are provided by NTUA. Energy average is the average value beetwen the two of NTUA.

<hr>

#### 4) Why do you need the target data? In your code it is January 31, but in a more general case it is the next day (tomorrow)?

#### Answer: is only to measure the quality

<hr>

#### 5) In DT dashboard we have also the Termal Sensation Vote chart. How and where can we get the data to show in this chart?

#### Answer: There are no Thermal Sensation votes at this time.

<hr>

#### 6) To get the next day's recommendations, do we need to retrain the forecasting model for sPMV values, or can we just use the model, already trained, to get the sPMV values for the next day and use those for optimization?

#### Answer: No. We can use the model already trained. They agree with the service process below.

just to be clear, our idea is: 

- during the night (e.g. at 01:00 AM):
    - **Download** yesterday's necessary sensor data from ISRAA(Domx) APIs (e.g. tr, rh_t, energy average)
    - **Calculate** the remaing values (e.g. hour, day of week, sPMV...) for yestarday.
    - **Add a row** for yestarday value in the dataset file (features_with_forecast.csv).
    - **Start** prediction using the model *final_model.h5* to estimate the values of forecasted sPMV and update the features_with_forecast.csv file.
    - **Run** pareto optimization to obtain tomorrow's s1, s2, s3 profiles and the summary.
- After the script has finished, the user views the data and recommendations for the current day in the DT.

<hr>

#### 7) Do we need to use all historical data to make a prediction for tomorrow, or is it sufficient to consider only recent data (e.g., the last 7 days, 15 days, 30 days, or 1 year)? 

For the recommendation we need all the historical data. Therefore we need to have and maintain the enteire dataset with all values until execution.

<hr>

#### 8) Why do we have data for 2025-01-31 in the features_with_forecast.csv file if we're supposed to make a prediction for that day (2025-01-31)? Shouldn't the last value in features_with_forecast.csv be for 2025-01-30 instead? When the data for 2025-01-31 is omitted, an error occurs. 

#### Answer: is only to measure the quality. in "pareto_optimization_runner.py:240" there is a split in historical and target data. The target one contains only the 31 january data.

<hr>

#### 9) How can I start optimization without target day?

#### Answer: they will do the modifications for us.

<hr>

#### 10) at running time we will have real data (t_h, energy_average, rh_r, forecasted_sPMV) for yesterday and forecasted data for today (forecasted_sPMV). Thus, we can perform pareto optimization on the historical data up to and including yesterday. In this way are the recommendations for today or for yesterday?

#### Answer: the recommendations are for today. We don't need to change nothing